In [ ]:
import io
import requests
from PIL import Image
import gradio as gr

# ─────────────────────────────────────────────────────────────────
# 1) 내 정보로 아래 두 변수를 수정하세요.
# ─────────────────────────────────────────────────────────────────
PREDICTION_KEY = "BBvYKDdr5RDpSMjG34Z2XXw3hLxzlAQkktCPXwHTLleSagQPHGg0JQQJ99BEACYeBjFXJ3w3AAAIACOGH9bC"
ENDPOINT_URL    = "https://7aiteam05cv-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/2ae6121f-4235-4dce-bf2a-fbace9444880/classify/iterations/Iteration12/image"
# ─────────────────────────────────────────────────────────────────


def predict_clean_heavy(img: Image.Image) -> dict:
    """
    Gradio로 입력받은 PIL 이미지를 JPEG 바이트로 변환해
    Custom Vision Prediction API에 POST 요청을 보냅니다.
    반환되는 JSON 안의 'predictions' 필드에서
    tagName(라벨)과 probability(확률)만 뽑아서
    {'clean': 확률, 'heavy': 확률} 형태의 dict를 리턴합니다.
    """
    # 1) PIL Image → JPEG 바이트 변환
    with io.BytesIO() as buffer:
        img.save(buffer, format="JPEG")
        image_bytes = buffer.getvalue()

    # 2) 요청 헤더 구성
    headers = {
        "Prediction-Key": PREDICTION_KEY,
        "Content-Type": "application/octet-stream"
    }

    # 3) API 호출 (POST)
    response = requests.post(
        ENDPOINT_URL,
        headers=headers,
        data=image_bytes
    )
    response.raise_for_status()  # 오류 시 예외 발생

    # 4) JSON 결과 파싱
    result = response.json()
    predictions = result.get("predictions", [])

    # 5) Gradio Label 컴포넌트에 맞게 {label:probability} dict 생성
    output = {}
    for pred in predictions:
        name = pred.get("tagName")
        prob = pred.get("probability", 0.0)
        # Gradio Label에 주려면 0~1 사이의 확률 값을 그대로 넘기면 됩니다.
        output[name] = prob

    return output


if __name__ == "__main__":

    demo = gr.Interface(
    fn=predict_clean_heavy,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=2, label="확률 (clean vs heavy)"),
    title="Custom Vision: Clean vs Heavy 분류",
    description="""
    웹캠으로 빗물받이 사진을 찍으면 Azure Custom Vision 이진 분류 모델에 보내서 
    'clean'과 'heavy' 각각의 확률을 실시간으로 보여줍니다.
    """,
    allow_flagging="never"
    )

    demo.launch(share=True)


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://25e5d76dbffe845179.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
import io
import requests
from PIL import Image
import gradio as gr

# ─────────────────────────────────────────────────────────────────
# 1) 아래 두 변수를 실제 값으로 수정하세요: (빗물받이 여부 판별 모델)
# ─────────────────────────────────────────────────────────────────
DETECTION_KEY = "5sloqMYWiZSpfCd7HZgQ9ZpfuQAXnRWwjSw648WjXGr5Fy5f7imcJQQJ99BFACYeBjFXJ3w3AAAIACOGBDCR"
DETECTION_ENDPOINT_URL = "https://7aiteam05ai012-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/f360eeb2-f8df-441e-8bed-f27d3ef1797a/detect/iterations/Iteration2/image"

# ─────────────────────────────────────────────────────────────────
# 2) 아래 두 변수를 실제 값으로 수정하세요: (오염도 분류 모델: Clean vs Heavy)
# ─────────────────────────────────────────────────────────────────
POLLUTION_KEY = "BBvYKDdr5RDpSMjG34Z2XXw3hLxzlAQkktCPXwHTLleSagQPHGg0JQQJ99BEACYeBjFXJ3w3AAAIACOGH9bC"
POLLUTION_ENDPOINT_URL = "https://7aiteam05cv-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/2ae6121f-4235-4dce-bf2a-fbace9444880/classify/iterations/Iteration12/image"


def predict_sequence(img: Image.Image) -> tuple[dict, dict, str]:
    """
    1) Gradio로 입력받은 PIL 이미지를 JPEG 바이트로 변환
    2) [빗물받이 여부 모델]에 POST 요청 -> 결과를 { 'drain':확률, 'not_drain':확률 }로 정리
    3) 'drain' 확률이 'not_drain'보다 높으면(=빗물받이 판정):
         -> [오염도 모델]에 POST 요청 -> 결과를 { 'clean':확률, 'heavy':확률 }로 정리
       아니면(=빗물받이가 아니면) 두 번째 모델 호출 생략
    4) 상태 메시지를 세 가지 경우로 분기하여 문자열로 생성
       - 빗물받이가 아님
       - 오염도(heavy) < 70%
       - 오염도(heavy) ≥ 70% & < 90%
       - 오염도(heavy) ≥ 90%
    5) Gradio에 (detection_dict, pollution_dict, status_text) 튜플로 반환
    """
    # ─────────────────────────────────────────────────────────────────
    # A) PIL Image → JPEG 바이트 변환 (공통)
    # ─────────────────────────────────────────────────────────────────
    with io.BytesIO() as buffer:
        img.save(buffer, format="JPEG")
        image_bytes = buffer.getvalue()

    # ─────────────────────────────────────────────────────────────────
    # B) 빗물받이 여부 모델 호출 (Detection)
    # ─────────────────────────────────────────────────────────────────
    headers_det = {
        "Prediction-Key": DETECTION_KEY,
        "Content-Type": "application/octet-stream"
    }
    response_det = requests.post(
        DETECTION_ENDPOINT_URL,
        headers=headers_det,
        data=image_bytes
    )
    response_det.raise_for_status()
    result_det = response_det.json()
    preds_det = result_det.get("predictions", [])

    # 출력 딕셔너리를 모두 소문자 키로 정리
    detection_dict = {}
    for pred in preds_det:
        name = pred.get("tagName", "")
        prob = pred.get("probability", 0.0)
        detection_dict[name.lower()] = prob
    # 예) detection_dict == { 'drain': 0.85, 'not_drain': 0.15 }

    # ─────────────────────────────────────────────────────────────────
    # C) 빗물받이 판정 기준: drain 확률 vs not_drain 확률
    #    (여기서는 더 높은 쪽을 선택)
    # ─────────────────────────────────────────────────────────────────
    drain_prob = detection_dict.get("street_drain", 0.0)
    not_drain_prob = detection_dict.get("unstreet_drain", 0.0)

    # 기본값: 오염도 모델 호출을 건너뛸 때 사용
    pollution_dict = { "clean": 0.0, "heavy": 0.0 }
    status_text = ""

    if drain_prob > not_drain_prob:
        # ─────────────────────────────────────────────────────────────────
        # D) 빗물받이(Drain)으로 판정되었으므로 오염도(클린/헤비) 모델 호출
        # ─────────────────────────────────────────────────────────────────
        headers_pol = {
            "Prediction-Key": POLLUTION_KEY,
            "Content-Type": "application/octet-stream"
        }
        response_pol = requests.post(
            POLLUTION_ENDPOINT_URL,
            headers=headers_pol,
            data=image_bytes
        )
        response_pol.raise_for_status()
        result_pol = response_pol.json()
        preds_pol = result_pol.get("predictions", [])

        # clean/heavy 확률을 소문자 키로 정리
        pollution_dict = {}
        for pred in preds_pol:
            name = pred.get("tagName", "")
            prob = pred.get("probability", 0.0)
            pollution_dict[name.lower()] = prob
        # 예) pollution_dict == { 'clean': 0.10, 'heavy': 0.90 }

        # ─────────────────────────────────────────────────────────────────
        # E) pollution_dict['heavy'] 값에 따라 상태 메시지 분기
        # ─────────────────────────────────────────────────────────────────
        heavy_prob = pollution_dict.get("heavy", 0.0)
        if heavy_prob >= 0.9:
            status_text = "🚨 Heavy 수준: 확률 ≥ 90% (매우 막힘)"
        elif heavy_prob >= 0.7:
            status_text = "⚠️ Heavy 수준: 확률 ≥ 70% (중간/높음 막힘)"
        elif heavy_prob >= 0.6:
            status_text = "✅ Heavy 확률 ≥ 60% (크게 막힌 상태 아님)"
        else:
            status_text = "✅ Clean"
    else:
        # ─────────────────────────────────────────────────────────────────
        # F) 빗물받이가 아닐 때: pollution_dict는 0.0 기본값 사용, 메시지 설정
        # ─────────────────────────────────────────────────────────────────
        status_text = "❌ 빗물받이가 아닙니다."

    # ─────────────────────────────────────────────────────────────────
    # G) Gradio로 반환: (빗물받이 판별 dict, 오염도 dict, 상태 메시지)
    # ─────────────────────────────────────────────────────────────────
    return detection_dict, pollution_dict, status_text


if __name__ == "__main__":
    demo = gr.Interface(
        fn=predict_sequence,
        inputs=gr.Image(type="pil"),
        outputs=[
            gr.Label(label="빗물받이 여부 (Drain vs NotDrain)"),
            gr.Label(label="오염도 (Clean vs Heavy)"),
            gr.Text(label="상태 메시지")
        ],
        title="Custom Vision: 빗물받이 여부 ➔ 오염도 순차 분류",
        description="""
        1. 먼저 업로드된 사진이 빗물받이인지 아닌지 Custom Vision 모델로 판정합니다.  
        2. '빗물받이(drain)'로 판정되면, 두 번째 모델을 호출하여 'clean' vs 'heavy' 오염도를 예측합니다.  
        3. 결과는 각각 확률과 상태 메시지로 보여줍니다.  
        - 빗물받이가 아닌 경우: 상태 메시지에 '❌ 빗물받이가 아닙니다.' 출력  
        - 빗물받이인 경우, heavy 확률에 따라 70%·90% 경계로 상태 메시지 분기  
        """,
        allow_flagging="never"
    )
    demo.launch(share=True)


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://dd3dcc1d9ae3cdecf0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 빗물 받이 확인 여부 + 빗물 받이 상태 + 위치 정보

In [6]:
import io
import requests
from PIL import Image, ExifTags
import gradio as gr

# ─────────────────────────────────────────────────────────────────
# 1) 빗물받이 여부 판별 모델 API 정보
# ─────────────────────────────────────────────────────────────────
DETECTION_KEY = "5sloqMYWiZSpfCd7HZgQ9ZpfuQAXnRWwjSw648WjXGr5Fy5f7imcJQQJ99BFACYeBjFXJ3w3AAAIACOGBDCR"
DETECTION_ENDPOINT_URL = "https://7aiteam05ai012-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/f360eeb2-f8df-441e-8bed-f27d3ef1797a/detect/iterations/Iteration2/image"

# ─────────────────────────────────────────────────────────────────
# 2) 오염도 분류 모델 API 정보
# ─────────────────────────────────────────────────────────────────
POLLUTION_KEY = "BBvYKDdr5RDpSMjG34Z2XXw3hLxzlAQkktCPXwHTLleSagQPHGg0JQQJ99BEACYeBjFXJ3w3AAAIACOGH9bC"
POLLUTION_ENDPOINT_URL = "https://7aiteam05cv-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/2ae6121f-4235-4dce-bf2a-fbace9444880/classify/iterations/Iteration12/image"

# ─────────────────────────────────────────────────────────────────
# GPS 추출 함수들
# ─────────────────────────────────────────────────────────────────
def get_decimal_from_dms(dms, ref):
    deg = float(dms[0])
    minu = float(dms[1])
    sec = float(dms[2])
    dec = deg + (minu / 60.0) + (sec / 3600.0)
    if ref in ('S', 'W'):
        dec = -dec
    return dec

def extract_gps(img_path):
    img = Image.open(img_path)
    exif = img._getexif()
    if not exif:
        return None

    gps_info = {}
    for tag_id, value in exif.items():
        tag = ExifTags.TAGS.get(tag_id)
        if tag == 'GPSInfo':
            for key, val in value.items():
                subtag = ExifTags.GPSTAGS.get(key)
                gps_info[subtag] = val

    required = ('GPSLatitudeRef', 'GPSLatitude', 'GPSLongitudeRef', 'GPSLongitude')
    if not all(k in gps_info for k in required):
        return None

    lat = get_decimal_from_dms(gps_info['GPSLatitude'], gps_info['GPSLatitudeRef'])
    lon = get_decimal_from_dms(gps_info['GPSLongitude'], gps_info['GPSLongitudeRef'])
    return (lat, lon)

# ─────────────────────────────────────────────────────────────────
# 메인 예측 함수 (파일 경로 기반)
# ─────────────────────────────────────────────────────────────────
def predict_sequence(image_path: str):
    try:
        # PIL 이미지 열기
        pil_img = Image.open(image_path)

        # A) 이미지 바이트 변환
        with open(image_path, "rb") as f:
            image_bytes = f.read()

        # B) 빗물받이 여부 예측
        headers_det = {
            "Prediction-Key": DETECTION_KEY,
            "Content-Type": "application/octet-stream"
        }
        response_det = requests.post(DETECTION_ENDPOINT_URL, headers=headers_det, data=image_bytes)
        response_det.raise_for_status()
        preds_det = response_det.json().get("predictions", [])
        detection_dict = {pred["tagName"].lower(): pred["probability"] for pred in preds_det}

        drain_prob = detection_dict.get("street_drain", 0.0)
        not_drain_prob = detection_dict.get("unstreet_drain", 0.0)

        pollution_dict = {"clean": 0.0, "heavy": 0.0}
        status_text = ""

        # C) 오염도 분류 (빗물받이일 경우만)
        if drain_prob > not_drain_prob:
            headers_pol = {
                "Prediction-Key": POLLUTION_KEY,
                "Content-Type": "application/octet-stream"
            }
            response_pol = requests.post(POLLUTION_ENDPOINT_URL, headers=headers_pol, data=image_bytes)
            response_pol.raise_for_status()
            preds_pol = response_pol.json().get("predictions", [])
            pollution_dict = {pred["tagName"].lower(): pred["probability"] for pred in preds_pol}

            heavy_prob = pollution_dict.get("heavy", 0.0)
            if heavy_prob >= 0.9:
                status_text = "🚨 Heavy 수준: 확률 ≥ 90% (매우 막힘)"
            elif heavy_prob >= 0.7:
                status_text = "⚠️ Heavy 수준: 확률 ≥ 70% (중간/높음 막힘)"
            elif heavy_prob >= 0.6:
                status_text = "✅ Heavy 확률 ≥ 60% (크게 막힌 상태 아님)"
            else:
                status_text = "✅ Clean"
        else:
            status_text = "❌ 빗물받이가 아닙니다."

        # D) GPS 추출
        try:
            coords = extract_gps(image_path)
            if coords:
                lat_lon_text = f"📍 위도: {coords[0]:.6f}, 경도: {coords[1]:.6f}"
            else:
                lat_lon_text = "📍 위치 정보 없음"
        except Exception as gps_err:
            lat_lon_text = f"📍 위치 정보 추출 실패: {gps_err}"

        return detection_dict, pollution_dict, status_text, lat_lon_text

    except Exception as e:
        return {"error": 1}, {"error": 1}, f"❌ 예외 발생: {str(e)}", "❌ 위치 정보 추출 실패"

# ─────────────────────────────────────────────────────────────────
# Gradio 인터페이스
# ─────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    demo = gr.Interface(
        fn=predict_sequence,
        inputs=gr.Image(type="filepath"),
        outputs=[
            gr.Label(label="빗물받이 여부 (Drain vs NotDrain)"),
            gr.Label(label="오염도 (Clean vs Heavy)"),
            gr.Text(label="상태 메시지"),
            gr.Text(label="GPS 위치 정보")
        ],
        title="Custom Vision: 빗물받이 여부 ➔ 오염도 ➔ 위치 정보",
        description="""
1. 업로드된 사진이 빗물받이인지 판단  
2. 빗물받이라면 오염도(Clean vs Heavy)를 예측  
3. JPEG의 EXIF 위치 정보가 있으면 위도/경도 출력  
""",
        allow_flagging="never"
    )
    demo.launch(share=True)


c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://2e6616778c4756c34e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\starlette\applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\sit